<a href="https://colab.research.google.com/github/suriyachaudary/recursive_LU_factorization/blob/main/Recursive_LU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recursive LU factorization

Reference [LU Factorization](https://netlib.org/utk/papers/factor/node7.html)

- [x] LU factorization for 2x2 matrix
- [x] Verify LU factorization result
- [x] Verify lower and upper triangular matrices
- [x] Extend LU factorization sub result
- [x] Rank of matrix
- [ ] Row permutation

In [1]:
import numpy as np

In [6]:
def unpack_result(result):
    P = result['P']
    L = result['L']
    E = result['E']
    U = result['U']
    return P, L, E, U

def extend_result(sub_result, shape):
    P, L, E, U = unpack_result(sub_result)
    P = np.append(P, np.zeros((1, P.shape[1])), axis = 0)
    P = np.append(P, np.identity(P.shape[0])[:,-1][:, np.newaxis], axis = 1)
    L = np.append(L, np.zeros((1, L.shape[1])), axis = 0)
    L = np.append(L, np.identity(L.shape[0])[:,-1][:, np.newaxis], axis = 1)
    E = np.append(E, np.zeros((1, E.shape[1])), axis = 0)
    E = np.append(E, np.identity(E.shape[0])[:,-1][:, np.newaxis], axis = 1)
    U = np.append(U, np.zeros((1, U.shape[1])), axis = 0)
    return {'P': P, 'L': L, 'E': E, 'U': U}

def complete_extended_sub_result(A, extended_sub_result):
    P, L, E, U = unpack_result(extended_sub_result)
    for col in range(min(A.shape[1], E.shape[1]-1)):
        row = E.shape[0]-1
        pivot = A[col, col]
        E_ = np.identity(E.shape[0])
        L_ = np.identity(L.shape[0])
        E_[row, col] = -A[row, col]/pivot
        L_[row, col] = -E_[row, col]
        U[row, :] = E_[row, :]@A[:E.shape[0], :]
        A[:U.shape[0], :U.shape[1]] = U

        E = E_@E
        L = L@L_

    return {'P': P, 'L': L, 'E': E, 'U': U}

def recursive_lu(A, result = {}):
    rows, cols = A.shape
    assert rows > 0 and cols > 0, "Rows and columns cannot be 0"

    P = np.identity(rows)
    L = np.identity(rows)
    E = np.identity(rows)

    if rows == 1:
        U = np.array(A)

    if rows == 2:
        E[1, 0] = -A[1, 0]/A[0, 0]
        L[1, 0] = -E[1, 0]
        U = E@A
    if rows > 2:
        sub_result = recursive_lu(A[0:2, :])

        while sub_result['E'].shape[0] < rows:
            extended_sub_result = extend_result(sub_result, A.shape)
            sub_result = complete_extended_sub_result(np.copy(A), extended_sub_result)
            P, L, E, U = unpack_result(sub_result)

    return {'P': P, 'L': L, 'E': E, 'U': U}

def rank(A):
    result = recursive_lu(A)
    return np.count_nonzero(np.diagonal(result['U']))

In [7]:
def verify_lu(A, result):
    assert A is not None and A.shape[0] > 0 and A.shape[1] > 0, "Input is empty array"
    assert result is not None \
        and result['P'].shape[0] > 0 and result['P'].shape[1] > 0 \
        and result['L'].shape[0] > 0 and result['L'].shape[1] > 0 \
        and result['U'].shape[0] > 0 and result['U'].shape[1] > 0, "Result is empty array"
    assert np.allclose(np.triu(result['L']), np.triu(np.identity(result['L'].shape[0]))), "L is not lower triangular"
    assert np.allclose(np.tril(result['U'], -1), np.tril(np.zeros(result['U'].shape), -1)), "U is not upper triangular"
    A_result = (result['P']@result['L'])@result['U']
    assert np.array_equal(A.shape, A_result.shape), "A.shape != A_result.shape"
    assert np.allclose(A, A_result), "A != PLU"
    print("Passed")

In [8]:
for rows in range(1, 10):
    for cols in range(1, 10):
        print(rows, cols)
        A = np.random.random((rows, cols))
        result = recursive_lu(A)
        print("Rank ", rank(A))
        verify_lu(A, result)

1 1
Rank  1
Passed
1 2
Rank  1
Passed
1 3
Rank  1
Passed
1 4
Rank  1
Passed
1 5
Rank  1
Passed
1 6
Rank  1
Passed
1 7
Rank  1
Passed
1 8
Rank  1
Passed
1 9
Rank  1
Passed
2 1
Rank  1
Passed
2 2
Rank  2
Passed
2 3
Rank  2
Passed
2 4
Rank  2
Passed
2 5
Rank  2
Passed
2 6
Rank  2
Passed
2 7
Rank  2
Passed
2 8
Rank  2
Passed
2 9
Rank  2
Passed
3 1
Rank  1
Passed
3 2
Rank  2
Passed
3 3
Rank  3
Passed
3 4
Rank  3
Passed
3 5
Rank  3
Passed
3 6
Rank  3
Passed
3 7
Rank  3
Passed
3 8
Rank  3
Passed
3 9
Rank  3
Passed
4 1
Rank  1
Passed
4 2
Rank  2
Passed
4 3
Rank  3
Passed
4 4
Rank  4
Passed
4 5
Rank  4
Passed
4 6
Rank  4
Passed
4 7
Rank  4
Passed
4 8
Rank  4
Passed
4 9
Rank  4
Passed
5 1
Rank  1
Passed
5 2
Rank  2
Passed
5 3
Rank  3
Passed
5 4
Rank  4
Passed
5 5
Rank  5
Passed
5 6
Rank  5
Passed
5 7
Rank  5
Passed
5 8
Rank  5
Passed
5 9
Rank  5
Passed
6 1
Rank  1
Passed
6 2
Rank  2
Passed
6 3
Rank  3
Passed
6 4
Rank  4
Passed
6 5
Rank  5
Passed
6 6
Rank  6
Passed
6 7
Rank  6
Passed
6 8
Rank  6
